In [56]:
import random
from collections import Counter

In [57]:
# Constants for ranks and suits
RANKS = "23456789TJQKA"
SUITS = "♠♥♦♣"

# Poker hand rankings
HAND_RANKS = {
    "High Card": 1,
    "One Pair": 2,
    "Two Pair": 3,
    "Three of a Kind": 4,
    "Straight": 5,
    "Flush": 6,
    "Full House": 7,
    "Four of a Kind": 8,
    "Straight Flush": 9,
    "Royal Flush": 10
}

class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
    
    def __repr__(self):
        return self.rank + self.suit

class Deck:
    def __init__(self):
        self.cards = [Card(r, s) for r in RANKS for s in SUITS]
        random.shuffle(self.cards)
    
    def deal(self, count):
        hand = self.cards[:count]
        self.cards = self.cards[count:]
        return hand

class PokerHand:
    def __init__(self, cards):
        self.cards = cards

    def rank(self):
        ranks = [card.rank for card in self.cards]
        suits = [card.suit for card in self.cards]
        rank_counts = Counter(ranks)
        is_flush = len(set(suits)) == 1
        is_straight = len(set(ranks)) == 5 and (max(map(RANKS.index, ranks)) - min(map(RANKS.index, ranks)) == 4)

        if is_straight and is_flush:
            return HAND_RANKS["Straight Flush"]
        elif rank_counts.most_common(1)[0][1] == 4:
            return HAND_RANKS["Four of a Kind"]
        elif len(rank_counts) == 2:
            return HAND_RANKS["Full House"]
        elif is_flush:
            return HAND_RANKS["Flush"]
        elif is_straight:
            return HAND_RANKS["Straight"]
        elif rank_counts.most_common(1)[0][1] == 3:
            return HAND_RANKS["Three of a Kind"]
        elif len([rank for rank, count in rank_counts.items() if count == 2]) == 2:
            return HAND_RANKS["Two Pair"]
        elif rank_counts.most_common(1)[0][1] == 2:
            return HAND_RANKS["One Pair"]
        return HAND_RANKS["High Card"]

def evaluate_winner(hands):
    winner = max(hands, key=lambda hand: hand.rank())
    return winner.rank(), hands.index(winner)



In [58]:
class EnhancedPokerGameMultiRound:
    def __init__(self, num_players, starting_balance):
        self.num_players = num_players
        self.starting_balance = starting_balance
        self.balances = [starting_balance] * num_players
        self.active_players = set(range(num_players))
        self.reset_for_new_round()

    def reset_for_new_round(self):
        self.deck = Deck()
        self.player_hands = [PokerHand(self.deck.deal(2)) for _ in range(self.num_players)]
        self.community_cards = self.deck.deal(5)
        self.pot = 0
        self.current_bet = 0

    def bet(self, player, amount):
        actual_bet = min(amount, self.balances[player])
        self.balances[player] -= actual_bet
        self.pot += actual_bet
        self.current_bet = max(self.current_bet, actual_bet)

    def call(self, player):
        # Assuming self.current_bet is the amount to call
        # and self.player_bets is a list tracking each player's bet in the round
        amount_to_call = self.current_bet - self.player_bets[player]
        actual_bet = min(amount_to_call, self.balances[player])
        self.balances[player] -= actual_bet
        self.pot += actual_bet
        self.player_bets[player] += actual_bet

    def raise_bet(self, player, amount):
        self.current_bet += amount
        self.bet(player, self.current_bet)

    def fold(self, player):
        self.active_players.remove(player)

    def showdown(self):
        if len(self.active_players) == 1:
            winner = next(iter(self.active_players))
            return winner, "Default Win"
        else:
            final_hands = [PokerHand(self.player_hands[i].cards + self.community_cards) for i in self.active_players]
            winning_rank, winning_player_idx = evaluate_winner(final_hands)
            winning_player = list(self.active_players)[winning_player_idx]
            return winning_player, winning_rank

    def play_round(self, user_player=0, stage="Pre-Flop"):
        self.current_bet = 0
        min_raise = 10  # Minimum raise amount
        player_bets = [0] * self.num_players  # Track how much each player has bet in this round
        last_raiser = None

        while True:
            round_complete = True
            for player in range(self.num_players):
                if player not in self.active_players or self.balances[player] <= 0:
                    continue  # Skip inactive players or those with no balance
                
                to_call = self.current_bet - player_bets[player]  # Amount needed to match the current bet
                if player == last_raiser and to_call == 0:
                    continue  # Skip if this player was the last to raise and no one else has raised since

                if to_call > 0:
                    round_complete = False  # If anyone needs to call or raise, the round is not complete
                
                if player == user_player:  # User's turn
                    print(f"\nPlayer {player + 1}, it's your turn.")
                    print(f"Your hand: {self.player_hands[player].cards}")
                    if stage != "Pre-Flop":
                        print(f"Community cards: {self.community_cards[:3 if stage == 'Flop' else 4 if stage == 'Turn' else 5]}")
                    print(f"Amount to call: {to_call}. Your current bet this round: {player_bets[player]}")
                    print(f"Total pot: {self.pot}. Current highest bet: {self.current_bet}")

                    if to_call >= self.balances[player]:  # All-in scenario
                        action = input("You don't have enough to call. Choose 'all-in' or 'fold': ").strip().lower()
                        while action not in ["all-in", "fold"]:
                            action = input("Invalid action. Choose 'all-in' or 'fold': ").strip().lower()
                    else:
                        action = input("Choose your action (call/raise/fold): ").strip().lower()
                        while action not in ["call", "raise", "fold"]:
                            action = input("Invalid action. Choose 'call', 'raise', or 'fold': ").strip().lower()

                    if action == "call":
                        self.call(player, to_call)
                        player_bets[player] += to_call
                    elif action == "raise":
                        raise_amount = max(min_raise, int(input("Enter the amount you want to raise: ")))
                        total_bet = to_call + raise_amount
                        self.raise_bet(player, total_bet)
                        player_bets[player] += total_bet
                        self.current_bet += raise_amount
                        last_raiser = player
                    elif action == "fold":
                        self.fold(player)
                    elif action == "all-in":
                        self.bet(player, self.balances[player])
                        player_bets[player] += self.balances[player]
                else:
                    # Simplified logic for non-user players; replace with more complex AI as needed
                    action = random.choice(["call", "raise", "fold"])
                    if action == "call":
                        self.call(player, to_call)
                        # player_bets[player] += to_call
                    elif action == "raise":
                        raise_amount = min_raise  # Simplified raise amount for non-user players
                        total_bet = to_call + raise_amount
                        self.raise_bet(player, total_bet)
                        player_bets[player] += total_bet
                        self.current_bet += raise_amount
                        last_raiser = player
                        round_complete = False  # Since there's a new raise, others need a chance to respond
                    elif action == "fold":
                        self.fold(player)
                    # Implement all-in for non-user players as needed

            if round_complete or len(self.active_players) <= 1:
                break  # End the betting round if everyone has responded to the latest bet or raise, or if the game is over

        return "continue" if len(self.active_players) > 1 else "exit"


    def play_game_indefinitely(self):
        round_count = 0
        while len({p for p in self.active_players if self.balances[p] > 0}) > 1:
            round_count += 1
            print(f"\nRound {round_count}: Starting")
            
            # Reset for a new round and deal two private cards to each player
            self.reset_for_new_round()
            print(f"Player hands at the start of round {round_count}:")
            for i, hand in enumerate(self.player_hands):
                if i in self.active_players:
                    print(f"Player {i + 1}: {hand.cards}")
            
            # Pre-flop betting round
            print("\nPre-Flop Betting Round:")
            if self.play_round(user_player=0) == "exit":
                print("Game terminated by user.")
                return
            print(f"Total pot after Pre-Flop: {self.pot}")

            # Reveal the flop (first three community cards)
            print("\nFlop:")
            print(self.community_cards[:3])
            print("Flop Betting Round:")
            if self.play_round(user_player=0) == "exit":
                print("Game terminated by user.")
                return
            print(f"Total pot after Flop: {self.pot}")

            # Reveal the turn (fourth community card)
            print("\nTurn:")
            print(self.community_cards[3])
            print("Turn Betting Round:")
            if self.play_round(user_player=0) == "exit":
                print("Game terminated by user.")
                return
            print(f"Total pot after Turn: {self.pot}")

            # Reveal the river (fifth community card)
            print("\nRiver:")
            print(self.community_cards[4])
            print("River Betting Round:")
            if self.play_round(user_player=0) == "exit":
                print("Game terminated by user.")
                return
            print(f"Total pot after River: {self.pot}")

            # Determine the winner and distribute the pot
            active_with_balance = {p for p in self.active_players if self.balances[p] > 0}
            if len(active_with_balance) > 0:
                winner, rank = self.showdown()
                self.balances[winner] += self.pot
                print(f"\nWinner of Round {round_count}: Player {winner + 1} with {list(HAND_RANKS.keys())[rank - 1]}")
                print(f"Total pot won: {self.pot}")
            self.active_players = active_with_balance

        # Announce the final winner or termination
        if len(active_with_balance) == 1:
            final_winner = next(iter(active_with_balance))
            print(f"\nGame Over. Final winner: Player {final_winner + 1} with balance: {self.balances[final_winner]}")
        else:
            print("Game Over. No final winner due to early termination or insufficient players.")


In [59]:
# Create and start the game
game = EnhancedPokerGameMultiRound(4, 100)
game.play_game_indefinitely()


Round 1: Starting
Player hands at the start of round 1:
Player 1: [9♠, K♠]
Player 2: [6♥, 8♠]
Player 3: [K♣, Q♣]
Player 4: [J♥, 4♦]

Pre-Flop Betting Round:

Player 1, it's your turn.
Your hand: [9♠, K♠]
Amount to call: 0. Your current bet this round: 0
Total pot: 0. Current highest bet: 0


TypeError: EnhancedPokerGameMultiRound.call() takes 2 positional arguments but 3 were given